# Google Oauth

In [9]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from datetime import datetime, timedelta


In [173]:
SCOPES = ["https://www.googleapis.com/auth/calendar"]
creds = None

if os.path.exists('./data/token.json'):
    creds = Credentials.from_authorized_user_file('./data/token.json', SCOPES)

# 자격 증명이 없거나 만료됐으면 새로 로그인
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # credentials.json 파일에서 클라이언트 비밀키 불러오기
        flow = InstalledAppFlow.from_client_secrets_file(
            './data/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json', SCOPES)
        creds = flow.run_local_server(port=0)

    # token.json에 저장
    with open('./data/token.json', 'w') as token:
        token.write(creds.to_json())

# CalendarToolkit Setting

In [4]:
from langchain_google_community import CalendarToolkit
from langchain_google_community.calendar.utils import (
    build_resource_service,
    get_google_credentials,
)

credentials = get_google_credentials(
    token_file="./data/token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="./data/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json",
)

api_resource = build_resource_service(credentials=credentials)
toolkit = CalendarToolkit(api_resource=api_resource)
toolkit

CalendarToolkit(api_resource=<googleapiclient.discovery.Resource object at 0x00000200755FEAB0>)

In [5]:
# tools = toolkit.get_tools()
# tools

# 캘린더 목록 조회

In [6]:
def list_calendars():
    calendar_list = api_resource.calendarList().list().execute()
    items = calendar_list.get('items', [])
    
    print("캘린더 목록:")
    for calendar in items:
        print(f"- {calendar.get('summary')} (ID: {calendar.get('id')})")
res = list_calendars()
res

캘린더 목록:
- 대한민국의 휴일 (ID: ko.south_korea#holiday@group.v.calendar.google.com)
- 가족 (ID: family13100623974177768096@group.calendar.google.com)
- jongbaekim0710@gmail.com (ID: jongbaekim0710@gmail.com)


# 캘린더 이벤트 추가

In [7]:
from datetime import datetime, timedelta
def add_meeting_event(calendar_id: str, start_datetime: datetime, summary: str = "회의"):
    end_datetime = start_datetime + timedelta(hours=1)

    event = {
        "summary": summary,
        "start": {
            "dateTime": start_datetime.isoformat(),
            "timeZone": "Asia/Seoul",  # 필요 시 변경
        },
        "end": {
            "dateTime": end_datetime.isoformat(),
            "timeZone": "Asia/Seoul",
        },
    }

    created_event = api_resource.events().insert(calendarId=calendar_id, body=event).execute()
    print(f"일정이 추가되었습니다: {created_event.get('htmlLink')}")

In [64]:
# calendar_id = "jongbaekim0710@gmail.com"  # 또는 사용자 캘린더 ID
# start_datetime = datetime(2025, 5, 22, 14, 0)  
# summary="기장 설계부 미팅"
# add_meeting_event(calendar_id=calendar_id, start_datetime=start_datetime, summary=summary)

# 캘린더 이벤트 아이디 조회

In [10]:
def event_dict(calendar_id: str, date: datetime):
    time_min = date.isoformat() + "Z"
    time_max = (date + timedelta(days=1)).isoformat() + "Z"

    events_result = api_resource.events().list(
        calendarId=calendar_id,
        timeMin=time_min,
        timeMax=time_max,
        singleEvents=True,
        orderBy="startTime"
    ).execute()
    
    events = events_result.get("items", [])
    results  = dict()
    for event in events:
        results[event.get('summary')] = event.get('id')

    return results

In [ ]:
# 사용 예시
# calendar_id = "jongbaekim0710@gmail.com"
# events = event_dict(calendar_id=calendar_id, date=datetime(2025, 5, 22))  # 해당 날짜의 이벤트 목록 확인
# events

# 특정 이벤트 수정

In [11]:
def update_event(calendar_id: str, event_id: str, new_summary=None, new_start=None, new_end=None):
    updated_fields = {}

    if new_summary:
        updated_fields["summary"] = new_summary
    if new_start and new_end:
        updated_fields["start"] = {
            "dateTime": new_start.isoformat(),
            "timeZone": "Asia/Seoul"
        }
        updated_fields["end"] = {
            "dateTime": new_end.isoformat(),
            "timeZone": "Asia/Seoul"
        }

    try:
        updated_event = api_resource.events().patch(
            calendarId=calendar_id,
            eventId=event_id,
            body=updated_fields
        ).execute()
        print(f"이벤트가 수정되었습니다: {updated_event.get('htmlLink')}")
    except Exception as e:
        print(f"이벤트 수정 중 오류 발생: {e}")

In [12]:
# calendar_id = "jongbaekim0710@gmail.com"
# event_id = event_dict["설계부 미팅"]

# new_title = "선장 설계부 미팅"
# new_start_time = datetime(2025, 5, 22, 16, 0)  # 오후 4시
# new_end_time = new_start_time + timedelta(hours=1)

# update_event(calendar_id, event_id, new_summary=new_title, new_start=new_start_time, new_end=new_end_time)

# 캘린더 이벤트 삭제

In [ ]:
def delete_event(calendar_id: str, event_id: str):
    try:
        api_resource.events().delete(calendarId=calendar_id, eventId=event_id).execute()
        print(f"이벤트(ID: {event_id})가 삭제되었습니다.")
    except Exception as e:
        print(f"이벤트 삭제 중 오류 발생: {e}")

In [34]:
# calendar_id = "jongbaekim0710@gmail.com"  # 또는 다른 캘린더 ID
# event_id = "tpf3ljtj2h5ar6mb238ptaltv0"  # 예: '5a4sk8s9d0fsdkf0sdg'
# delete_event(calendar_id, event_id)

# Structured Output

In [ ]:
import os
import json
from datetime import datetime
from typing import List, Union
from typing import Optional
from groq import Groq
from pydantic import BaseModel, ValidationError, field_validator

# 환경 변수 로드
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY environment variable is missing")

# 출력 스키마 정의
class EventIdInfo(BaseModel):
    event_id: str

class ScheduleInfo(BaseModel):
    job_type: str
    old_time: Optional[datetime] = None
    old_info: Optional[str] = None
    new_time: Optional[datetime] = None
    new_info: Optional[str] = None
    del_time: Optional[datetime] = None
    del_info: Optional[str] = None

# Groq 클라이언트 초기화
client = Groq(api_key=GROQ_API_KEY)

def get_event_id(user_input:str, event_dict: dict) -> str:
    system_prompt = f"""
    You are an expert id finder.
    Blow Ref Events are consist of title and id (key, value) pair dictionary.
    Find the event that is most similar to the input value among Ref Events presented and return the most similar item of that events.
    
    <Ref Events>
    {event_dict}

    <User Input>
    {user_input}

    Output MUST be valid JSON containing array of item
    """

    try:
        response = client.chat.completions.create(
            model="gemma2-9b-it",  # JSON 출력에 더 적합한 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            response_format={"type": "json_object"},
            temperature=0.0,  # 약간의 창의성 허용
            max_tokens=100
        )

        json_str = response.choices[0].message.content        
        # print(json_str)
        try:
            parsed = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON response: {e}")

        # 단일 객체인 경우 리스트로 변환
        if isinstance(parsed, dict):
            parsed = [parsed]

        return parsed if parsed else None

    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return None

def is_new_event(user_input: str, event_list: List[str]) -> bool:
    current_date = datetime.now().strftime("%Y-%m-%d")
    
    # 기존 이벤트 리스트를 프롬프트에 포함
    event_list_text = "\n".join(f"- {event}" for event in event_list)
    
    system_prompt = f"""
    You are an expert calendar assistant. Today's date is {current_date} (Asia/Seoul timezone).
    You will determine if the user's input contains a new event **not already present** in the existing schedule.

    <Instructions>
    Compare the user's input to the following list of existing schedule items.
    If the user's input describes an event that already exists (even approximately), respond with:
        {{ "is_new": false }}
    If the user's input describes a completely new event (not in the list), respond with:
        {{ "is_new": true }}
    Return only the JSON object and nothing else.

    <Existing Events>
    {event_list_text}
    """

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            response_format={"type": "json_object"},
            temperature=0.3,
            max_tokens=100
        )

        json_str = response.choices[0].message.content

        try:
            result = json.loads(json_str)
            return result.get("is_new", True)  # 기본값은 True (새 이벤트)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON response: {e}")

    except Exception as e:
        print(f"Error during event check: {str(e)}")
        return True  # 에러 발생 시 기본적으로 새 이벤트라고 간주

def process_schedule_request(user_input: str) -> Union[List[ScheduleInfo], None]:
    current_date = datetime.now().strftime("%Y-%m-%d")
    system_prompt = f"""
    You are an expert calendar assistant. Today's date is {current_date} (Asia/Seoul timezone).
    If the year is not mentioned, just assume this year is 2025.
    
    <Review instructions>
    1. define job type among New, Update, Delete based on the user input
    2. accroding to the job type, review step by step and extract below schedule information.

    - old_time : canceled schedule time (only when job_type is Update)
    - old_info : canceled schedule information (only when job_type is Update)
    - new_time : new or current meeting or event time (if not specified, maintain old_time)
    - new_info : new or current schedule information (if not specified, maintain old_info and Do not include time info)
    - del_time : meeting or event time to be removed 
    - del_info : meeting or event information to be removed  

    Do not include TxInfo in datetime value.
    Generate final answer as a concise and compact 2~3 keywords in Korean Language
    Output MUST be valid JSON containing either a single object or array of objects."""

    try:
        response = client.chat.completions.create(
            model="gemma2-9b-it",  # JSON 출력에 더 적합한 모델
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            response_format={"type": "json_object"},
            temperature=0.5,  # 약간의 창의성 허용
            max_tokens=500
        )

        json_str = response.choices[0].message.content        
        # JSON 파싱
        try:
            parsed = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON response: {e}")

        # 단일 객체인 경우 리스트로 변환
        if isinstance(parsed, dict):
            parsed = [parsed]

        # 유효성 검사
        validated = []
        for idx, item in enumerate(parsed, 1):
            try:
                validated.append(ScheduleInfo(**item))
                print(f"Item {idx} validated successfully")
            except ValidationError as ve:
                print(f"Validation error in item {idx}: {ve}")

        return validated if validated else None

    except Exception as e:
        print(f"Error processing request: {str(e)}")
        return None

# 테스트

In [ ]:
def main(user_input:str, calendar_id: str="jongbaekim0710@gmail.com"):
    # Auth
    SCOPES = ["https://www.googleapis.com/auth/calendar"]
    creds = None

    if os.path.exists('./data/token.json'):
        creds = Credentials.from_authorized_user_file('./data/token.json', SCOPES)

    # 자격 증명이 없거나 만료됐으면 새로 로그인
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # credentials.json 파일에서 클라이언트 비밀키 불러오기
            flow = InstalledAppFlow.from_client_secrets_file(
                './data/client_secret_39562377782-nge5sdugil9eurkbgn54temjtgq06tbh.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)

    # token.json에 저장
    with open('./data/token.json', 'w') as token:
        token.write(creds.to_json())

    # process_schedule_request (type 등 필요 정보 추출)
    res = process_schedule_request(user_input=user_input)
    print(res)
    results = {'job_type': res[0].job_type,
               'old_time': res[0].old_time,
               'old_info': res[0].old_info,
               'new_time': res[0].new_time,
               'new_info': res[0].new_info,
               'del_time': res[0].del_time,
               'del_info': res[0].del_info
               }
    print(results)

    if results['job_type'] == "New":
        print("일정 추가")
        # 해당 일정이 있는지 확인
        date_only = datetime(results['new_time'].year, results['new_time'].month, results['new_time'].day)
        print(date_only)
        events = event_dict(calendar_id=calendar_id, date=date_only)  # 해당 날짜의 이벤트 목록 확인
        events_list = list(events.keys())
        print(f">>> 조회일자 이벤트 리스트 : {events_list}")
        new_or_not = is_new_event(user_input=user_input, event_list=events_list)
        print(f">>> 신규 일정 여부 : {new_or_not}")
        if new_or_not:
            add_meeting_event(calendar_id=calendar_id, start_datetime=results['new_time'], summary=results['new_info'])
            print(f"일정 등록 완료 - {results["new_time"]} - {results["new_info"]}")
        else: print("이미 해당 일정이 등록되어 있습니다.")

    elif results['job_type'] == "Update":
        print("일정 수정")
        # 해당 일정이 있는지 확인
        date_only = datetime(results['old_time'].year, results['old_time'].month, results['old_time'].day)
        print(date_only)
        events = event_dict(calendar_id=calendar_id, date=date_only)  # 해당 날짜의 이벤트 목록 확인
        events_list = list(events.keys())
        print(f">>> 조회일자 이벤트 리스트 : {events_list}")
        new_or_not = is_new_event(user_input=results['old_info'], event_list=events_list)
        print(f">>> 신규 일정 여부 : {new_or_not}")

        if new_or_not == False:  # 삭제대상 올드 인포가 이벤트 리스트에서 조회되는 경우
            # 삭제
            event_id = get_event_id(user_input=user_input, event_dict=events)
            print(f">>> 삭제대상 이벤트 ID :{event_id[0]['items'][0]['id']}")
            delete_event(calendar_id=calendar_id, event_id=event_id[0]['items'][0]['id'])
            #수정 등록
            add_meeting_event(calendar_id=calendar_id, start_datetime=results['new_time'], summary=results['new_info'])
            print(f"일정 등록 완료 - {results["new_time"]} - {results["new_info"]}")
        
        else: print("조회대상 이벤트에 수정 일정이 없습니다.")

    elif results['job_type'] == "Delete":
        print("일정 삭제")
        # 해당 일정이 있는지 확인
        date_only = datetime(results['del_time'].year, results['del_time'].month, results['del_time'].day)
        print(date_only)
        events = event_dict(calendar_id=calendar_id, date=date_only)  # 해당 날짜의 이벤트 목록 확인
        print(f">>> 조회일자 이벤트 dict : {events}")
        event_id = get_event_id(user_input=user_input, event_dict=events)
        print(f">>> 삭제대상 이벤트 ID :{event_id[0]['items'][0]['id']}")
        delete_event(calendar_id=calendar_id, event_id=event_id[0]['items'][0]['id'])


    else : 
        print("Job Type를 확인할 수 없습니다.")


In [181]:
user_input = "5월 23일 상선기본설계부 미팅 삭제"
main(user_input=user_input)

Item 1 validated successfully
[ScheduleInfo(job_type='Delete', old_time=None, old_info=None, new_time=None, new_info=None, del_time=datetime.datetime(2025, 5, 23, 0, 0), del_info='상선기본설계부 미팅')]
{'job_type': 'Delete', 'old_time': None, 'old_info': None, 'new_time': None, 'new_info': None, 'del_time': datetime.datetime(2025, 5, 23, 0, 0), 'del_info': '상선기본설계부 미팅'}
일정 삭제
2025-05-23 00:00:00
>>> 조회일자 이벤트 dict : {'5월 23일 오전 10시 기본설계1부 미팅': '7dq96tclcl235oc939siustpk4'}
>>> 삭제대상 이벤트 ID :7dq96tclcl235oc939siustpk4
이벤트(ID: 7dq96tclcl235oc939siustpk4)가 삭제되었습니다.
